# Phase 2: Find Processing Techniques
## Train and Visualize Custom CNN Models 
### Import Libraries and Scripts

In [2]:
%reload_ext autoreload
%autoreload 2

In [14]:
# import functions 
from Utils.preporcessing_utils import data_loading 
from Utils.preporcessing_utils import labels_encoding
from Utils.preporcessing_utils import split_data
from Utils.preporcessing_utils import ablation
from Utils.preporcessing_utils_Copy1 import preprocess_locally
from Utils.models_utils import Basic_Custom_CNN
from Utils.evaluation_utils import Evaluation
from Utils.visualizations_utils import Visualization
from Utils.save_data_utils import Save_Data
from Utils.run_model_utils import run_model

In [4]:
# import libraries
import json 
import os
import cv2

### Data Preparation and Basic Preprocessing

In [5]:
# Loads data
train_df, test_df = data_loading("train_full.csv", "test_full.csv")

In [6]:
# Create dataframe and transform(encodes) pathology labels
train_df, test_df = labels_encoding(train_df, test_df)

In [7]:
train_df.columns

Index(['image_id', 'image_type', 'image_path', 'series_uid', 'subject_id',
       'study_uid', 'breast_density', 'breast_side', 'image_view',
       'abnormality_type', 'pathology', 'split', 'label'],
      dtype='object')

In [8]:
# Split data
train_data, val_data, test_data = split_data(train_df, test_df, 0.11)

Train set: 1889 cases, 70.35 %
Validation set: 234 cases, 8.72 %
Test set: 562 cases, 20.93 %


In [9]:
train_10 = train_data[:10]
val_data_10 = val_data[:10]
test_data_10 = test_data[:10]

### Create Preprocessing Groups by Using Ablation

In [10]:
# create groups using ablation
options = ['apply_background_removal',
           'apply_crop',
           'apply_noise_reduction',
           'apply_contrast_enhancement',
           'apply_edge_enhancement',
           'apply_lbp_texturizer']

In [11]:
# create group of techniques to try
techniques_groups = ablation(options)

In [12]:
techniques_groups["Baseline Basic Preporcessing"]

{'apply_background_removal': False,
 'apply_crop': False,
 'apply_noise_reduction': False,
 'apply_contrast_enhancement': False,
 'apply_edge_enhancement': False,
 'apply_lbp_texturizer': False}

### Baseline model with ImageGenerator
This models uses the custom CNN with 10 epochs and the tf.ImageDataGenerator image iterator. <br>
The preprocessing was done Using the ImageGenerator during training. <br><br> 
The following model call will not run because it is not connected the image iterator funtion anymore becaue the function was updated. <br> 
This output was kept for reference only. 
If needed, please see **run_model_utils.py** for running instructions.  

In [12]:
# runs baseline model with an image dataset generator 
run_model((train_data, val_data, test_data), techniques_groups, epochs=10, change="Baseline(tf.ImageDataGenerator)")

Training Custom10 - Baseline Basic Preporcessing - Baseline(tf.ImageDataGenerator)
Found 1889 validated image filenames.
Found 234 validated image filenames.
Found 562 validated image filenames.


I0000 00:00:1756665577.419168   33308 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1756665588.778667   33462 service.cc:148] XLA service 0x739a240099a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756665588.778734   33462 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-08-31 14:39:48.797362: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1756665588.887976   33462 cuda_dnn.cc:529] Loaded cuDNN version 90501
E0000 00:00:1756665591.911711   33462 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756665592.022091   33462 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


 1/60 ━━━━━━━━━━━━━━━━━━━━ 9:03 9s/step - accuracy: 0.5000 - loss: 0.6927

...

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 297s 5s/step - accuracy: 0.5272 - loss: 0.6929 - val_accuracy: 0.5171 - val_loss: 0.6927
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 280s 5s/step - accuracy: 0.5092 - loss: 0.6932 - val_accuracy: 0.5171 - val_loss: 0.6931
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 280s 5s/step - accuracy: 0.5023 - loss: 0.6931 - val_accuracy: 0.5171 - val_loss: 0.6931
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 280s 5s/step - accuracy: 0.5022 - loss: 0.6930 - val_accuracy: 0.5171 - val_loss: 0.6926
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 281s 5s/step - accuracy: 0.5224 - loss: 0.6927 - val_accuracy: 0.5171 - val_loss: 0.6930
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 290s 5s/step - accuracy: 0.5211 - loss: 0.6927 - val_accuracy: 0.5171 - val_loss: 0.6930
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 292s 5s/step - accuracy: 0.5177 - loss: 0.6930 - val_accuracy: 0.5171 - val_loss: 0.6933
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 281s 5s/step - accuracy: 0.5164 - loss: 0.6926 - val_accuracy: 0.5171 - v

/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 322s 5s/step - accuracy: 0.4842 - loss: 0.6932 - val_accuracy: 0.5171 - val_loss: 0.6930
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 344s 6s/step - accuracy: 0.4977 - loss: 0.6932 - val_accuracy: 0.5171 - val_loss: 0.6930
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 336s 6s/step - accuracy: 0.5133 - loss: 0.6929 - val_accuracy: 0.5171 - val_loss: 0.6928
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 319s 5s/step - accuracy: 0.5165 - loss: 0.6927 - val_accuracy: 0.5171 - val_loss: 0.6930
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 302s 5s/step - accuracy: 0.5216 - loss: 0.6929 - val_accuracy: 0.5171 - val_loss: 0.6926
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 311s 5s/step - accuracy: 0.5262 - loss: 0.6921 - val_accuracy: 0.5171 - val_loss: 0.6929
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 293s 5s/step - accuracy: 0.4927 - loss: 0.6934 - val_accuracy: 0.5171 - val_loss: 0.6929
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 309s 5s/step - accuracy: 0.5159 - loss: 0.6928 - val_accuracy: 0.5171 - v

/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 320s 5s/step - accuracy: 0.4873 - loss: 0.6932 - val_accuracy: 0.5171 - val_loss: 0.6931
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 310s 5s/step - accuracy: 0.4752 - loss: 0.6932 - val_accuracy: 0.5171 - val_loss: 0.6929
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 322s 5s/step - accuracy: 0.5305 - loss: 0.6925 - val_accuracy: 0.5171 - val_loss: 0.6927
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 314s 5s/step - accuracy: 0.5188 - loss: 0.6927 - val_accuracy: 0.5171 - val_loss: 0.6928
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 308s 5s/step - accuracy: 0.5194 - loss: 0.6927 - val_accuracy: 0.5171 - val_loss: 0.6928
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 313s 5s/step - accuracy: 0.5261 - loss: 0.6924 - val_accuracy: 0.5171 - val_loss: 0.6926
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 304s 5s/step - accuracy: 0.5055 - loss: 0.6933 - val_accuracy: 0.5171 - val_loss: 0.6927
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 317s 5s/step - accuracy: 0.4969 - loss: 0.6936 - val_accuracy: 0.5171 - v

## Search Preprocessing Technique Using Custom CNN

In [11]:
architecture_settings = {
    "iteration 1": {
        "epochs":10,
        "layers": [32, 64, 128],
        "activation": 'relu',
        "dropout": None
    },
    "iteration 1": {
        "epochs":10,
        "layers": [32, 64, 128],
        "activation": 'relu',
        "dropout": 0.2
    }
}

### Baseline model with Image tf.Dataset Generator
This models uses the custom CNN with 10 epochs and the tf.Dataset image iterator  

In [15]:
# runs baseline model with an image dataset generator 
run_model((train_10, val_data_10, test_data_10), techniques_groups, with_preprocess=False, epochs=10, change="Baseline(tf.Dataset)")

Training Custom10 - Baseline Basic Preporcessing - Baseline(tf.Dataset)


I0000 00:00:1757188448.683965   96984 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │       115,201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 207,873 (812.00 KB)

 Trainable params: 207,873 (812.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


2025-09-06 15:54:10.649013: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: /Preprocessed_Images/custom_baseline_basic_preporcessing/1.3.6.1.4.1.9590.100.1.2.323499822612107717622923360172400112625_full.png; No such file or directory
2025-09-06 15:54:10.649070: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: /Preprocessed_Images/custom_baseline_basic_preporcessing/1.3.6.1.4.1.9590.100.1.2.81539754211044334004264470202062485069_full.png; No such file or directory
2025-09-06 15:54:10.649110: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: /Preprocessed_Images/custom_baseline_basic_preporcessing/1.3.6.1.4.1.9590.100.1.2.245398881011643929625650254320825052239_full.png; No such file or directory
2025-09-06 15:54:10.649127: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at whole_file_read_ops

NotFoundError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) NOT_FOUND:  /Preprocessed_Images/custom_baseline_basic_preporcessing/1.3.6.1.4.1.9590.100.1.2.81539754211044334004264470202062485069_full.png; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
  (1) NOT_FOUND:  /Preprocessed_Images/custom_baseline_basic_preporcessing/1.3.6.1.4.1.9590.100.1.2.81539754211044334004264470202062485069_full.png; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_one_step_on_iterator_1806]

## Add Layers to Basic Model

## Add Dropout 

### Iteration 1: Basic CNN Architecture 

#### Training

#### Visualization and Evaluation

In [ ]:
# retrieve models data 
with open('../Outputs/models_data.json', 'r') as file:
        data = json.load(file)

In [ ]:
# shows confusion matrix
vis = Visualization()